In [ ]:
from moviepy.editor import *
import pandas as pd
def cut(file):
    print('读取视频')
    clip2 = VideoFileClip(file)
    print('读取音频')
    audioclip = clip2.audio
    soundarray = audioclip.to_soundarray()
    soundarray = pd.DataFrame(soundarray)
    soundarray = abs(soundarray)
    print('读取音量低的部分')
    yuzhi = 0.02
    gap = 0.01
    a = pd.DataFrame(list(soundarray[soundarray[0]<yuzhi].index/44100)).round(2)
    a['count'] = 1
    a = (a.groupby(0)['count'].sum()/(44100*gap))
    rate = 0.8
    a = a[a<rate]
    a = list(a.index)
    a = pd.DataFrame(a,columns = ['x'])
    a['x_1'] = a.shift(-1)
    a['flag'] = a['x_1']-a['x']
    a = a[a['flag']>gap*1.1]
    ls = []
    for i in range(len(a)-1):
        ls.append([a.iloc[i]['x_1'],a.iloc[i+1]['x']])
    print('裁剪')
    clips = []
    for i in ls:
        if i[0]<i[1]:
            clips.append(clip2.subclip(i[0],i[1]))
    print('渲染合成')
    finalclip = concatenate_videoclips(clips)
    finalclip.write_videofile('cliped'+file)

In [ ]:
import os
filelist = os.listdir()
for file in filelist:
    if file.split('.')[1]=='mp4':
        print(file)
        cut(file)

In [ ]:
cut('测试视频.mp4')